In [1]:
import cv2
import numpy as np
import gradio as gr

def sky_detection(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Define the blue color range in RGB
    lower_blue = np.array([0, 80, 120]) 
    upper_blue = np.array([255, 255, 255]) 

    # Create a mask for the aky color range in the RGB image
    sky_mask = cv2.inRange(image_rgb, lower_blue, upper_blue)

    # Find the edge and remove the edge from the sky
    edges = cv2.Canny(image_rgb,100,100)
    inverted_edges = cv2.bitwise_not(edges)
    combined_mask = cv2.bitwise_and(sky_mask, inverted_edges)

    # Define a kernel size
    kernel_size = 20 
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    combined_mask_morpho = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)

    # Create an image to apply the mask to
    colored_sky = np.zeros_like(image_rgb)
    colored_sky[np.where(combined_mask_morpho == 255)] = [255, 255, 0]  # Color the sky yellow in RGB format

    # Merge the original RGB image with the colored sky
    result_rgb = cv2.addWeighted(image_rgb, 0.8, colored_sky, 1, 0)

    return result_rgb

iface = gr.Interface(fn=sky_detection, inputs="file", outputs="image")
iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://c8879f533216340efa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
